In [149]:
from skimage import io
from skimage import transform
import numpy as np
import random
import math

# Funcion which creates array of puzzle pieces
# From a given picture

def make_puzzle(image_path, n_rows, n_cols):
  
  image = io.imread(image_path) # Image loading

  size = image.shape # Image dimensions
  size = min(size[0], size[1])
  
  image = image[0:size, 0:size] # Resize image on minimum shape

  step = math.floor(size / n_rows)
  
  pieces = []

  for i in range(n_rows):
    for j in range(n_cols):
      pieces.append(np.array(image[j*step: (j+1)*step-1, i*step : (i+1)*step-1]))

  random.shuffle(pieces)

  for x in pieces:
    i = random.randint(0, 4)
    transform.rotate(x, i*90)

  return pieces

pieces = make_puzzle("lion.jpg", 5, 5)
print(pieces)

[array([[[163, 124,  69],
        [175, 136,  81],
        [165, 124,  70],
        ...,
        [132,  94,  45],
        [149, 112,  67],
        [160, 123,  79]],

       [[141, 101,  50],
        [149, 109,  57],
        [151, 110,  58],
        ...,
        [123,  85,  36],
        [157, 121,  73],
        [163, 124,  81]],

       [[119,  81,  36],
        [138,  97,  51],
        [146, 104,  54],
        ...,
        [110,  70,  19],
        [142, 104,  57],
        [156, 118,  73]],

       ...,

       [[136,  93,  40],
        [129,  88,  36],
        [129,  89,  40],
        ...,
        [124, 104,  80],
        [118,  94,  66],
        [124,  95,  63]],

       [[144,  91,  47],
        [132,  81,  36],
        [144,  97,  53],
        ...,
        [ 79,  63,  47],
        [ 76,  57,  40],
        [ 80,  57,  39]],

       [[147,  92,  53],
        [144,  91,  51],
        [149, 100,  59],
        ...,
        [ 57,  43,  30],
        [ 50,  34,  21],
        [ 51,  31,  22]